In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install kaggle pandas

Note: you may need to restart the kernel to use updated packages.


In [8]:
import zipfile

In [9]:
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall()

In [10]:
import os

In [11]:
import pandas as pd

In [12]:
# 2.NLP Component (Text Analysis) start here with loading and preprocessing the text data

In [13]:
#Loading the Dataset
file_path = "dataset.csv"
data = pd.read_csv(file_path)
print(data.head())

                                               Image  \
0  [[[-0.5507521  -0.6045698  -0.6229138 ]\n  [-0...   
1  [[[-0.5480964  -0.5235891  -0.76556504]\n  [-0...   
2  [[[0.16979091 0.29803866 0.3613662 ]\n  [0.199...   
3  [[[-0.24860182 -0.35181937 -0.4667918 ]\n  [-0...   
4  [[[ 0.20944206  0.12295917  0.06715906]\n  [ 0...   

                                                Text Sentiment  
0  I am feeling positive today because I see Tibe...  POSITIVE  
1  I am feeling negative today because I see cric...  NEGATIVE  
2  I am feeling positive today because I see Cock...  POSITIVE  
3  I am feeling negative today because I see Grif...  POSITIVE  
4  I am feeling positive today because I see logg...  POSITIVE  


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#Downloading NLTK resources that I need
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Joshua.Turnbull\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joshua.Turnbull\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
#Loading text data dataset
text_data = data['Text']

In [38]:
image_paths = data["Image"].tolist()

In [16]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Joshua.Turnbull\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [17]:
def preprocess_text(text):
    #Applying lowercasing
    text = text.lower()
    
    #Applying Tokenization
    tokens = word_tokenize(text)
    
    #Applying stopword removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    
    return filtered_tokens

# Applying preprocessing to the text data
processed_text = text_data.apply(preprocess_text)
print(processed_text.head())

0    [feeling, positive, today, see, tibetan, terri...
1    [feeling, negative, today, see, cricket, feeli...
2    [feeling, positive, today, see, cocker, spanie...
3    [feeling, negative, today, see, griffon, bruxe...
4    [feeling, positive, today, see, loggerhead, se...
Name: Text, dtype: object


In [18]:
#Starting to extract features using BERT

In [19]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install torch

In [22]:
from transformers import BertTokenizer, BertModel

#Loading BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [25]:
#Extracting BERT Features
def get_bert_embeddings(text):
    #Tokenizing the text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Getting BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    #Using the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

#Applying BERT feature extraction
bert_features = text_data.apply(get_bert_embeddings)

In [24]:
#Building a Sentiment Classification Model

In [27]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 8.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.1 MB 6.5 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.1 MB 7.3 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.1 MB 6.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.1 MB 6.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.1 MB 7.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   -- ------------------------------------- 2.1/41.2 MB 13.1 MB/s eta 0:00:03
   ---- ----------------------------------- 4.5/41.2 MB 12.2 MB/s eta 0:00:04
   ----- ---------------------------------- 5.5/41.2 MB 9.1 MB/s eta 0:00:04
   ------

In [29]:
#Preparing data for modelling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

labels = data['Sentiment']

#Encoding labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

#Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_features.tolist(), encoded_labels, test_size=0.2, random_state=42)

In [30]:
#Training Sentiment Classification Model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Training a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

#Predicting and evaluating the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.9055944055944056


In [31]:
#3. Computer Vision Component (Image Analysis)

In [33]:
#Importing necessary libraries
import os
import numpy as np
from PIL import Image
import torch

In [34]:
pip install torch torchvision

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 16.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [35]:
from torchvision import transforms

In [36]:
#Defining preprossesing steps
preprocess = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])
#Resizing images to 224x224 pixels
#Converting image to PyTorch tensor
#Normalizing using ImageNet standards

In [41]:
#Loading and preprocessing images
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    return image

#Preprocessing images
images = [load_and_preprocess_image(image_path) for image_path in image_paths]
images = torch.stack(images)  
#Stacking images into a single tensor

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Joshua.Turnbull\\[[[-0.5507521  -0.6045698  -0.6229138 ]\n  [-0.63850087 -0.66662765 -0.6978787 ]\n  [-0.6789291  -0.66646045 -0.6680623 ]\n  ...\n  [-0.6993981  -0.7002809  -0.69596666]\n  [-0.61803526 -0.6082668  -0.60642767]\n  [-0.5558948  -0.57046324 -0.59524053]]\n\n [[-0.5584087  -0.5619373  -0.5744659 ]\n  [-0.60510385 -0.5802763  -0.5907305 ]\n  [-0.61694497 -0.55589324 -0.59315443]\n  ...\n  [-0.71316266 -0.7144597  -0.65562713]\n  [-0.62904453 -0.59384894 -0.5559391 ]\n  [-0.6673803  -0.667725   -0.6671463 ]]\n\n [[-0.55703044 -0.5463458  -0.5191123 ]\n  [-0.53970224 -0.49998084 -0.5225193 ]\n  [-0.55531585 -0.5167261  -0.5316111 ]\n  ...\n  [-0.6966244  -0.7041687  -0.6425341 ]\n  [-0.53042525 -0.5079934  -0.46229053]\n  [-0.46439797 -0.5104388  -0.50135607]]\n\n ...\n\n [[ 0.7684314   0.8806956   0.9020777 ]\n  [ 0.55004394  0.8354144   0.92739576]\n  [ 0.2053559   0.6938815   0.89285916]\n  ...\n  [ 0.514

In [42]:
#Using a pre-trained CNN (ResNet) for feature extraction

In [43]:
#Importing and loading model
from torchvision import models

model = models.resnet50(pretrained=True)
model.eval()  
#Setting the model to evaluation mode

C:\Users\Joshua.Turnbull\AppData\Local\miniconda3\envs\pydata-book\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Joshua.Turnbull\AppData\Local\miniconda3\envs\pydata-book\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Joshua.Turnbull/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [50]:
#Extracting features using ResNet
# Remove the final layer to extract features
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

def extract_features(images):
    with torch.no_grad():
        features = feature_extractor(images)
    return features.squeeze()

#Extracting features from the preprocessed images
image_features = extract_features(image)
print(image_features.shape)  
#Checking the shape of the feature vectors

NameError: name 'image' is not defined

In [52]:
#Building a sentiment classification model for image data

In [54]:
#Preparing data for modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Example of sentiment labels
labels = np.array([0, 1])

#Encoding labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_features.numpy(), encoded_labels, test_size=0.2, random_state=42)

NameError: name 'image_features' is not defined

In [55]:
#Taining the model using logistic rgeression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Training a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

#Predicting and evaluating the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.9055944055944056


In [56]:
#Fusion and Final Classification

In [57]:
#Concatenating feature vectors
import numpy as np

#Converting text and image features to numpy arrays
text_features = np.array(text_features.tolist())
image_features = image_features.numpy()

#Concatenating text and image features
combined_features = np.concatenate((text_features, image_features), axis=1)

NameError: name 'text_features' is not defined

In [59]:
#Preparing data for modeling
from sklearn.model_selection import train_test_split

#Splitting combined features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, encoded_labels, test_size=0.2, random_state=42)

NameError: name 'combined_features' is not defined

In [61]:
#Defining and training the model
import torch
import torch.nn as nn
import torch.optim as optim

#Defining a simple fully connected neural network
class SentimentClassifier(nn.Module):
    def __init__(self, input_dim):
        super(SentimentClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2) 
        )

    def forward(self, x):
        return self.fc(x)

#Initializing the model
input_dim = X_train.shape[1]
model = SentimentClassifier(input_dim)

#Defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Converting data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

#Training the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

AttributeError: 'list' object has no attribute 'shape'

In [62]:
#Evaluating the final model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Evaluating the model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = torch.argmax(y_pred_tensor, dim=1).numpy()

#Calculating performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')


AttributeError: 'LogisticRegression' object has no attribute 'eval'